In [ ]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import numpy as np
warnings.filterwarnings("ignore")
plt.style.use('ggplot')
pd.set_option('display.max_columns', None)
import plotly.graph_objects as go
import random

from dash import Dash, html, dcc
from jupyter_dash import JupyterDash
from dash.dependencies import Output, Input
from dash.exceptions import PreventUpdate

In [ ]:
data = pd.read_csv('Listing_of_Active_Businesses_20231016.csv')

In [ ]:
data.shape

In [ ]:
data.info()

In [ ]:
data.columns

In [ ]:
data.drop('LOCATION ACCOUNT #', axis=1, inplace=True)

In [ ]:
data.isna().sum()

In [ ]:
data['DBA NAME'].fillna('Unknown', inplace=True)

In [ ]:
#Dropped data where street address or is null

data.drop(data.loc[data['STREET ADDRESS'].isna()].index, inplace=True)

In [ ]:
data.loc[data['CITY'].isna()]['ZIP CODE'].unique()

In [ ]:
data['CITY'].fillna('Unknown', inplace=True)

In [ ]:
data.loc[data['ZIP CODE'].str.endswith('-')]['LOCATION'].value_counts()

In [ ]:
online_companies = data.loc[data["LOCATION"] == '(0.0, 0.0)']

In [ ]:
data.drop(['LOCATION DESCRIPTION', 'MAILING ADDRESS', 'MAILING CITY', 'MAILING ZIP CODE'], axis=1, inplace=True)

In [ ]:
data.isna().sum()

In [ ]:
#Droping the 'LOCATION END DATE' since the dataset is all about active business 

data.drop('LOCATION END DATE', axis=1, inplace=True)

In [ ]:
data['NAICS'].fillna('Unknown', inplace=True)

In [ ]:
data['PRIMARY NAICS DESCRIPTION'].fillna('Unknown', inplace=True)

In [ ]:
#Datetime value

data['LOCATION START DATE'] = pd.to_datetime(data['LOCATION START DATE'])

In [ ]:
#Location - LON/LAT

data['LOCATION'] = data['LOCATION'].str.replace('(', '').str.replace(')', '')

In [ ]:
location = data['LOCATION'].str.split(',', expand=True)

location.columns = ['LAT', 'LON']

In [ ]:
data = pd.concat([
    data.drop('LOCATION', axis=1),
    location], axis=1)

In [ ]:
#Duplicates

data.duplicated().sum()

data.drop_duplicates(inplace=True)

In [ ]:
data.duplicated(subset='BUSINESS NAME').sum()

In [ ]:
data[data.duplicated(subset='BUSINESS NAME')]

In [ ]:
data.reset_index(drop=True, inplace=True)

In [ ]:
data.head()

In [ ]:
# PRIMARY NAICS DESCRIPTION categorization

In [ ]:
data['PRIMARY NAICS DESCRIPTION'].unique()

In [ ]:
data['PRIMARY NAICS DESCRIPTION'] = data['PRIMARY NAICS DESCRIPTION'].str.replace('  ', ' ')

In [ ]:
category_dict = {
    'Educational services (including schools, colleges, & universities)': 'Education and Training',
    'Other financial investment activities (including investment advice)': 'Financial Services',
    'Independent artists, writers, & performers': 'Arts and Entertainment',
    'Limited-service eating places': 'Food and Dining',
    'Janitorial services': 'Professional and Business Services',
    'Clothing accessories stores': 'Retail and Shopping',
    'Single Family Housing Construction (1997 NAICS)': 'Construction and Building Services',
    'Offices of all other miscellaneous health practitioners': 'Healthcare and Medical Services',
    'Jewelry stores': 'Retail and Shopping',
    'Apparel mfg.': 'Manufacturing and Production',
    'Legal services': 'Professional and Business Services',
    'Reupholstery & furniture repair': 'Other Services',
    'All other personal services': 'Other Services',
    'Unknown': 'Miscellaneous',
    "Flower, nursery stock, & florists' supplies": 'Agriculture and Farming',
    'Lessors of real estate (including mini warehouses & self-storage units)': 'Real Estate and Property Management',
    'Motion picture & video industries (except video rental)': 'Arts and Entertainment',
    'Other direct selling establishments (including door-to-door retailing, frozen food plan providers, party plan merchandisers, & coffee-break service providers)': 'Transportation and Logistics',
    'Medical Marijuana Collective': 'Healthcare and Medical Services',
    'Performing arts companies': 'Arts and Entertainment',
    'Other services to buildings & dwellings': 'Other Services',
    'Home furnishings stores': 'Retail and Shopping',
    'All other professional, scientific, & technical services': 'Professional and Business Services',
    'Automotive mechanical & electrical repair & maintenance': 'Transportation and Logistics',
    'Electrical & electronic goods': 'Manufacturing and Production',
    'Engineering services': 'Professional and Business Services',
    'Camera & photographic supplies stores': 'Retail and Shopping',
    'Management, scientific, & technical consulting services': 'Professional and Business Services',
    'Offices of dentists': 'Healthcare and Medical Services',
    'Beauty salons': 'Healthcare and Medical Services',
    'Other consumer goods rental': 'Retail and Shopping',
    'Travel arrangement & reservation services': 'Transportation and Logistics',
    'Other amusement & recreation services (including golf courses, skiing facilities, marinas, fitness centers, bowling centers, skating rinks, miniature golf courses)': 'Arts and Entertainment',
    'Photographic services': 'Arts and Entertainment',
    'Child day care services': 'Healthcare and Medical Services',
    'Carpet & upholstery cleaning services': 'Professional and Business Services',
    'Full-service restaurants': 'Food and Dining',
    'Sound recording industries': 'Arts and Entertainment',
    'All other specialty trade contractors': 'Construction and Building Services',
    'Glass and Glazing Contractors (1997 NAICS)': 'Construction and Building Services',
    'Other transit & ground passenger transportation': 'Transportation and Logistics',
    'Nursing & residential care facilities': 'Healthcare and Medical Services',
    'Offices of mental health practitioners (except physicians)': 'Healthcare and Medical Services',
    'Offices of physicians (except mental health specialists)': 'Healthcare and Medical Services',
    'Other personal & household goods repair & maintenance': 'Other Services',
    'Specialized design services (including interior, industrial, graphic, & fashion design)': 'Professional and Business Services',
    'Plumbing, Heating, and Air-Conditioning Contractors (1997 NAICS)': 'Construction and Building Services',
    'Apparel, piece goods, & notions': 'Retail and Shopping',
    'Scientific research & development services': 'Professional and Business Services',
    'Books, periodicals, & newspapers': 'Retail and Shopping',
    'Drywall, Plastering, Acoustical, and Insulation Contractors (1997 NAICS)': 'Construction and Building Services',
    'Traveler accommodation (including hotels, motels, & bed & breakfast inns)': 'Transportation and Logistics',
    'Support activities for transportation (including motor vehicle towing)': 'Transportation and Logistics',
    'Fabricated metal product mfg.': 'Manufacturing and Production',
    'Lumber & other construction materials': 'Construction and Building Services',
    'Employment services': 'Professional and Business Services',
    'Medical & diagnostic laboratories': 'Healthcare and Medical Services',
    'Outpatient care centers': 'Healthcare and Medical Services',
    'Manufacturing and Industrial Building Construction (1997 NAICS)': 'Construction and Building Services',
    'Landscaping services': 'Construction and Building Services',
    'Cosmetics, beauty supplies, & perfume stores': 'Retail and Shopping',
    'Nail salons': 'Healthcare and Medical Services',
    'Commercial and Institutional Building Construction (1997 NAICS)': 'Construction and Building Services',
    'Other insurance related activities': 'Legal and Financial Services',
    'Other accounting services': 'Professional and Business Services',
    'Barber shops': 'Healthcare and Medical Services',
    'Jewelry, watch, precious stone, & precious metals': 'Retail and Shopping',
    'All other miscellaneous store retailers (including tobacco, candle, & trophy shops)': 'Retail and Shopping',
    'Other automotive repair & maintenance (including oil change & lubrication shops & car washes)': 'Transportation and Logistics',
    'Real estate property managers': 'Real Estate and Property Management',
    'Roofing, Siding, and Sheet Metal Contractors (1997 NAICS)': 'Construction and Building Services',
    'Payroll services': 'Professional and Business Services',
    'Other miscellaneous nondurable goods': 'Manufacturing and Production',
    'Office administrative services': 'Professional and Business Services',
    'Lawn & garden equipment & supplies stores': 'Retail and Shopping',
    'Commercial & industrial machinery & equipment rental & leasing': 'Transportation and Logistics',
    'Offices of real estate agents & brokers': 'Real Estate and Property Management',
    'Other miscellaneous mfg.': 'Manufacturing and Production',
    'Consumer electronics & appliances rental': 'Retail and Shopping',
    'General freight trucking, local': 'Transportation and Logistics',
    'Shoe stores': 'Retail and Shopping',
    'Other miscellaneous durable goods': 'Manufacturing and Production',
    'Promoters of performing arts, sports, & similar events': 'Arts and Entertainment',
    'Other activities related to real estate': 'Real Estate and Property Management',
    'Home health care services': 'Healthcare and Medical Services',
    'Offices of physical, occupational & speech therapists, & audiologists': 'Healthcare and Medical Services',
    'Drinking places (alcoholic beverages)': 'Food and Dining',
    'Beer, wine & liquor stores': 'Retail and Shopping',
    'Investigation & security services': 'Professional and Business Services',
    'Vending machine operators': 'Retail and Shopping',
    'Office supplies & stationery stores': 'Retail and Shopping',
    "Women's clothing stores": 'Retail and Shopping',
    'Art dealers': 'Arts and Entertainment',
    'Unclassified establishments (unable to classify)': 'Miscellaneous',
    'Musical instrument & supplies stores': 'Retail and Shopping',
    'Toy & hobby goods & supplies': 'Retail and Shopping',
    'Other food mfg. (including coffee, tea, flavoring, & seasonings)': 'Manufacturing and Production',
    'Family clothing stores': 'Retail and Shopping',
    'Other personal care services (including diet & weight reducing centers)': 'Healthcare and Medical Services',
    'Grocery stores (including supermarkets & convenience stores without gas)': 'Retail and Shopping',
    'Landscape architecture services': 'Construction and Building Services',
    'Professional & commercial equipment & supplies': 'Retail and Shopping',
    'Electrical Contractors (1997 NAICS)': 'Construction and Building Services',
    'Telecommunications': 'Technology and Telecommunications',
    'Internet publishing & broadcasting': 'Technology and Telecommunications',
    'Other Clothing Stores': 'Retail and Shopping',
    'General merchandise stores': 'Retail and Shopping',
    'Fruit & vegetable markets': 'Agriculture and Farming',
    'Agents & managers for artists, athletes, entertainers, & other public figures': 'Arts and Entertainment',
    'Textile product mills': 'Manufacturing and Production',
    'Computer systems design & related services': 'Technology and Telecommunications',
    'Computer & software stores': 'Retail and Shopping',
    'Household appliance stores': 'Retail and Shopping',
    'Couriers & messengers': 'Transportation and Logistics',
    'Computer & electronic product mfg.': 'Manufacturing and Production',
    'Waste management & remediation services': 'Professional and Business Services',
    'Sporting goods stores': 'Retail and Shopping',
    'Insurance agencies & brokerages': 'Legal and Financial Services',
    'Medical equipment & supplies mfg.': 'Manufacturing and Production',
    'Special food services (including food service contractors & caterers)': 'Food and Dining',
    'Multifamily Housing Construction (1997 NAICS)': 'Construction and Building Services',
    'Electronic shopping': 'Retail and Shopping',
    'Support activities for forestry': 'Transportation and Logistics',
    'Bakeries & tortilla mfg.': 'Food and Dining',
    'Warehousing & storage (except leases of mini warehouses & self-storage units)': 'Transportation and Logistics',
    'Tax preparation services': 'Professional and Business Services',
    'Drafting services': 'Professional and Business Services',
    'Floor Laying and Other Floor Contractors (1997 NAICS)': 'Construction and Building Services',
    'Used car dealers': 'Retail and Shopping',
    'Other building materials dealers': 'Retail and Shopping',
    'Used merchandise stores': 'Retail and Shopping',
    'Radio, television, & other electronics stores': 'Retail and Shopping',
    'Veterinary services': 'Healthcare and Medical Services',
    'Commercial & industrial machinery & equipment (except automotive & electronic) repair & maintenance': 'Professional and Business Services',
    'Tobacco & tobacco products': 'Retail and Shopping',
    'Furniture & home furnishing': 'Retail and Shopping',
    'Wholesale trade agents & brokers': 'Retail and Shopping',
    'Water Well Drilling Contractors (1997 NAICS)': 'Construction and Building Services',
    'Plastics & rubber products mfg.': 'Manufacturing and Production',
    'Internet service providers': 'Technology and Telecommunications',
    'Sporting & recreational goods & supplies': 'Retail and Shopping',
    'Individual & family services': 'Healthcare and Medical Services',
    'Printing & related support activities': 'Professional and Business Services',
    'Advertising & related services': 'Professional and Business Services',
    'Religious, grantmaking, civic, professional, & similar organizations': 'Nonprofit and Religious Organizations',
    'Poured concrete foundation & structure contractors': 'Construction and Building Services',
    'Machinery, equipment, & supplies': 'Manufacturing and Production',
    'Gasoline stations (including convenience stores with gas)': 'Retail and Shopping',
    'Specialized freight trucking (including household moving vans)': 'Transportation and Logistics',
    'Investment bankers & securities dealers': 'Legal and Financial Services',
    'Community food & housing, & emergency & other relief services': 'Nonprofit and Religious Organizations',
    'Other health & personal care stores': 'Retail and Shopping',
    'Activities related to credit intermediation (including loan brokers)': 'Legal and Financial Services',
    'Painting and Wall Covering Contractors (1997 NAICS)': 'Construction and Building Services',
    'Florists': 'Retail and Shopping',
    'Beverage & tobacco product mfg.': 'Manufacturing and Production',
    'Gift, novelty, & souvenir stores': 'Retail and Shopping',
    'Rooming & boarding houses': 'Hospitality and Lodging',
    "Men's clothing stores": 'Retail and Shopping',
    'Meat markets': 'Food and Dining',
    'Pharmacies & drug stores': 'Retail and Shopping',
    'Hobby, toy, & game stores': 'Retail and Shopping',
    'Architectural services': 'Professional and Business Services',
    'Tile & terrazzo contractors': 'Construction and Building Services',
    'Data processing, hosting, & related services': 'Technology and Telecommunications',
    'Electrical equipment, appliance, & component mfg.': 'Manufacturing and Production',
    'Recyclable materials': 'Manufacturing and Production',
    'Home & garden equipment & appliance repair & maintenance': 'Professional and Business Services',
    'Coin-operated laundries & drycleaners': 'Professional and Business Services',
    'Manufacture (mobile) home dealers': 'Real Estate and Property Management',
    'Offices of chiropractors': 'Healthcare and Medical Services',
    'Other information services (including news syndicates and libraries)': 'Technology and Telecommunications',
    'Automotive body, paint, interior, & glass repair': 'Transportation and Logistics',
    'Petroleum & petroleum products': 'Manufacturing and Production',
    'Mail-order houses': 'Retail and Shopping',
    'Parking lots & garages': 'Transportation and Logistics',
    "Children's & infants' clothing stores": 'Retail and Shopping',
    'Drycleaning & laundry services (except coin-operated) (including laundry & drycleaning drop-off & pickup sites)': 'Professional and Business Services',
    'Furniture stores': 'Retail and Shopping',
    'Electronic & precision equipment repair & maintenance': 'Professional and Business Services',
    'Telephone call centers (including telephone answering services & telemarketing bureaus)': 'Professional and Business Services',
    'Footwear & leather goods repair': 'Retail and Shopping',
    'Publishing industries (except Internet)': 'Arts and Entertainment',
    'Paper & paper products': 'Manufacturing and Production',
    'Grocery & related products': 'Retail and Shopping',
    'Carpentry Contractors (1997 NAICS)': 'Construction and Building Services',
    'Offices of certified public accountants': 'Professional and Business Services',
    'All other motor vehicle dealers': 'Retail and Shopping',
    'Facilities support (management) services': 'Professional and Business Services',
    'Oil & gas extraction': 'Energy and Utilities',
    'Translation & interpretation services': 'Professional and Business Services',
    'New car dealers': 'Retail and Shopping',
    'Hospitals': 'Healthcare and Medical Services',
    'Support activities for animal production (including farriers)': 'Agriculture and Farming',
    'Other business support services (including repossession services, court reporting, & stenotype services)': 'Professional and Business Services',
    'General rental centers': 'Retail and Shopping',
    'Transportation equipment mfg.': 'Manufacturing and Production',
    'Automotive parts, accessories, & tire stores': 'Retail and Shopping',
    'Taxi & limousine service': 'Transportation and Logistics',
    'Formal wear & costume rental': 'Retail and Shopping',
    'Textile mills': 'Manufacturing and Production',
    'Building inspection services': 'Construction and Building Services',
    'Motor vehicle & motor vehicle parts & supplies': 'Retail and Shopping',
    'Securities brokers': 'Legal and Financial Services',
    'Masonry and Stone Contractors (1997 NAICS)': 'Construction and Building Services',
    'Excavation Contractors (1997 NAICS)': 'Construction and Building Services',
    "Drugs & druggists' sundries": 'Retail and Shopping',
    'Other building equipment contractors': 'Construction and Building Services',
    'Market research & public opinion polling': 'Professional and Business Services',
    'Beer, wine, & distilled alcoholic beverage': 'Food and Dining',
    'Electronic auctions': 'Retail and Shopping',
    'Business service centers (including private mail centers & copy shops)': 'Professional and Business Services',
    'Other support services (including packaging & labeling services, & convention & trade show organizers)': 'Professional and Business Services',
    'Pet care (except veterinary) services': 'Healthcare and Medical Services',
    'Spectator sports (including professional sports clubs & racetrack operations)': 'Arts and Entertainment',
    'Optical goods stores': 'Retail and Shopping',
    'Wood product mfg.': 'Manufacturing and Production',
    'Document preparation services': 'Professional and Business Services',
    'Depository credit intermediation (including commercial banking, savings institutions, & credit unions)': 'Legal and Financial Services',
    'Offices of real estate appraisers': 'Real Estate and Property Management',
    'Museums, historical sites, & similar institutions': 'Arts and Entertainment',
    'Offices of optometrists': 'Healthcare and Medical Services',
    'Fruit & vegetable preserving & speciality food mfg.': 'Agriculture and Farming',
    'Interurban & rural bus transportation': 'Transportation and Logistics',
    'Hardware, & plumbing & heating equipment & supplies': 'Retail and Shopping',
    'Machinery mfg.': 'Manufacturing and Production',
    'Linen & uniform supply': 'Retail and Shopping',
    'Testing laboratories': 'Professional and Business Services',
    'Other ambulatory health care services (including ambulance services, blood, & organ banks)': 'Healthcare and Medical Services',
    'Vocational rehabilitation services': 'Healthcare and Medical Services',
    'Other Heavy and Civil Engineering Construction': 'Construction and Building Services',
    'Bridge and Tunnel Construction (1997 NAICS)': 'Construction and Building Services',
    'Furniture & related product mfg.': 'Manufacturing and Production',
    'Sewing, needlework, & piece goods stores': 'Retail and Shopping',
    'Prerecorded tape, compact disc, & record stores': 'Retail and Shopping',
    'Primary metal mfg.': 'Manufacturing and Production',
    'Photofinishing': 'Arts and Entertainment',
    'Scenic & sightseeing transportation': 'Transportation and Logistics',
    'Collection agencies': 'Professional and Business Services',
    'Surveying & mapping (except geophysical) services': 'Professional and Business Services',
    'Charter bus industry': 'Transportation and Logistics',
    'Automotive equipment rental & leasing': 'Transportation and Logistics',
    'Power and Communication Transmission Line Construction (1997 NAICS)': 'Construction and Building Services',
    'Paint & wallpaper stores': 'Retail and Shopping',
    'Chemical & allied products': 'Manufacturing and Production',
    'Offices of podiatrists': 'Healthcare and Medical Services',
    'Hardware stores': 'Retail and Shopping',
    'Home centers': 'Retail and Shopping',
    'Metal & mineral (except petroleum)': 'Manufacturing and Production',
    'Book stores': 'Retail and Shopping',
    'Sugar & confectionery product mfg.': 'Manufacturing and Production',
    'Other chemical products & preparation mfg.': 'Manufacturing and Production',
    'Land subdivision': 'Real Estate and Property Management',
    'Recreational vehicle dealers (including motor home & travel trailer dealers)': 'Retail and Shopping',
    'Nondepository credit intermediation (including sales financing & consumer lending)': 'Legal and Financial Services',
    'Structural Steel Erection Contractors (1997 NAICS)': 'Construction and Building Services',
    'Cellular and Other Wireless Telecommunications': 'Technology and Telecommunications',
    'Telecommunications Resellers': 'Technology and Telecommunications',
    'Funeral homes & funeral services': 'Healthcare and Medical Services',
    'Pet & pet supplies stores': 'Retail and Shopping',
    'Paper mfg.': 'Manufacturing and Production',
    'Exterminating & pest control services': 'Professional and Business Services',
    'Commodity contracts dealers': 'Legal and Financial Services',
    'Business to business electronic markets': 'Retail and Shopping',
    'Basic chemical mfg.': 'Manufacturing and Production',
    'Water, Sewer, and Pipeline Construction (1997 NAICS)': 'Construction and Building Services',
    'Gambling industries': 'Arts and Entertainment',
    'Highway and Street Construction (1997 NAICS)': 'Construction and Building Services',
    'Animal Slaughtering and Processing': 'Agriculture and Farming',
    'Farm product raw materials': 'Agriculture and Farming',
    'Luggage & leather goods stores': 'Retail and Shopping',
    'Paint, varnish, & supplies': 'Retail and Shopping',
    'Fuel dealers': 'Retail and Shopping',
    'Fish & seafood markets': 'Food and Dining',
    'Credit bureaus': 'Professional and Business Services',
    'School & employee bus transportation': 'Transportation and Logistics',
    'Security Systems Services (except locksmiths)': 'Professional and Business Services',
    'Amusement parks & arcades': 'Arts and Entertainment',
    'Farm supplies': 'Agriculture and Farming',
    'Video tape & disc rental': 'Arts and Entertainment',
    'Utilities': 'Energy and Utilities',
    'Motorcycle dealers': 'Retail and Shopping',
    'Other direct selling establishments (including door-to-door retailing, frozen food plan providers, party plan merchandisers, & coffee-break service providers)': 'Retail and Shopping',
'Other amusement & recreation services (including golf courses, skiing facilities, marinas, fitness centers, bowling centers, skating rinks,\nminiature golf courses)': 'Arts and Entertainment',
    'Geophysical surveying & mapping services': 'Professional and Business Services',
    'Securities & commodity exchanges': 'Legal and Financial Services',
    'Other amusement & recreation services (including golf courses, skiing facilities, marinas, fitness centers, bowling centers, skating rinks, miniature golf\ncourses)': 'Arts and Entertainment',
    'Petroleum & coal products mfg.': 'Manufacturing and Production',
    'Cemeteries & crematories': 'Healthcare and Medical Services',
    'Boat dealers': 'Retail and Shopping',
    'Commodity contracts brokers': 'Legal and Financial Services',
    'Paging': 'Technology and Telecommunications'
}


In [ ]:
data['CATEGORY'] = data['PRIMARY NAICS DESCRIPTION'].map(category_dict)

In [ ]:
data['CATEGORY'].value_counts()

In [ ]:
data.head()

In [ ]:
# Using Zip code with and the to find the active business in the city of LA 

In [ ]:
zip_code = pd.read_csv('zip.csv').drop('Unnamed: 0', axis=1)

zip_code['Zip Code'] = zip_code['Zip Code'].astype('str')

In [ ]:
zip_code

In [ ]:
zip1 = data['ZIP CODE'].str.split('-', expand=True)

In [ ]:
for x in zip_code['Zip Code']:
    df = zip1.loc[zip1[0].isin(zip_code['Zip Code'])]

In [ ]:
zip_dict = zip_code.set_index('Zip Code')['AREA NAME'].to_dict()

len(zip_dict)

In [ ]:
df.drop([1, 2], axis=1, inplace=True)

In [ ]:
df['City Name'] = df[0].map(zip_dict)

In [ ]:
la_zip = df.set_index(0)['City Name'].to_dict()

In [ ]:
data['ZIP CODE'] = zip1[0]

In [ ]:
la_data = data.loc[data['ZIP CODE'].isin(df[0])]

In [ ]:
la_data['CITY'] = la_data['ZIP CODE'].map(la_zip)

In [ ]:
la_data.reset_index(drop=True, inplace=True)

In [ ]:
la_data.head()

In [ ]:
la_data['CITY'].unique()

In [ ]:
la_data = la_data.drop(la_data.loc[la_data['CITY'] == 'Devonshire'].index).reset_index(drop=True)

In [ ]:
la_data['CITY'].value_counts()

In [ ]:
#Active Business Cnt in Cities in LA
px.bar(
    la_data['CITY'].value_counts().reset_index(), 
    y='index',
    x='CITY',
    color='index',
    text_auto=True,
    labels={'index':'City', 'CITY':'Active Business Cnt'},
    title='Active Business Cnt in Cities in LA'
).update_traces(showlegend=False)

In [ ]:
#City and business Category analysis

In [ ]:
la_data.groupby('CITY')['CATEGORY'].value_counts().unstack()

In [ ]:
pd.DataFrame(la_data.groupby('CITY')['CATEGORY'].value_counts()).rename(columns={'CATEGORY':'Cnt'})\
           .reset_index()

In [ ]:
app = JupyterDash(__name__)
app.layout = html.Div([
    html.H3('City Selection:'),
    dcc.Dropdown(id='dropdown', 
                options=la_data['CITY'].unique(), 
                value='West LA'),
    dcc.Graph(id='visual')
])


@app.callback(Output('visual', 'figure'), Input('dropdown', 'value'))

def plot(city):
    
    df = pd.DataFrame(la_data.groupby('CITY')['CATEGORY'].value_counts()).rename(columns={'CATEGORY':'Cnt'})\
           .reset_index()
    
    fig = px.bar(
        df.loc[df['CITY']==city], 
        y='CATEGORY',
        x='Cnt',
        color='CATEGORY',
        text_auto=True,
        title=f'Active Businesses Cnt in {city}'
    ).update_traces(showlegend=False)
    
    return fig

if __name__ == '__main__':
    app.run_server(debug=True, mode='inline', port=6019)

In [ ]:
#Calcucate Avg Years of history for each businesses in different city

In [ ]:
la_data['TODAY'] = '2023-01-01'

la_data['TODAY'] = pd.to_datetime(la_data['TODAY'])

In [ ]:
la_data['YEARS'] = round((la_data['TODAY'] - la_data['LOCATION START DATE']).dt.days / 365.25)

In [ ]:
la_data.drop(['TODAY'], axis=1, inplace=True)

In [ ]:
la_data.groupby(['CITY', 'CATEGORY'])['YEARS'].mean().unstack()

In [ ]:
la_data.groupby(['CITY', 'CATEGORY'])['YEARS'].mean().reset_index()

In [ ]:
app = JupyterDash(__name__)
app.layout = html.Div([
    html.H3('City Selection:'),
    dcc.Dropdown(id='dropdown1', 
                options=la_data['CITY'].unique(), 
                value='West LA'),
    html.Br(),
    dcc.Graph(id='visual')
])


@app.callback(Output('visual', 'figure'), 
              Input('dropdown1', 'value'))

def plot(city):
    
    df = la_data.groupby(['CITY', 'CATEGORY'])['YEARS'].mean().reset_index()\
                .sort_values('YEARS', ascending=False)
    
    fig = px.bar(
        df.loc[df['CITY']==city], 
        y='CATEGORY',
        x='YEARS',
        color='CATEGORY',
        text_auto=True,
        title=f'Avg Years of History of Business Category in {city}'
    ).update_traces(showlegend=False)
    
    return fig

if __name__ == '__main__':
    app.run_server(debug=True, mode='inline', port=6020)

In [ ]:
#Histograms

In [ ]:
for x in la_data['CITY'].unique():
    fig = px.histogram(la_data.loc[la_data['CITY'] == x],
                      x=['YEARS'],
                      title=x).update_traces(showlegend=False)
    fig.show()

In [ ]:
la_data['CATEGORY'].isna().sum()

In [ ]:
la_data['LOCATION START DATE'] = la_data['LOCATION START DATE'].astype('str')

In [ ]:
la_data['LOCATION START DATE'] = la_data['LOCATION START DATE'].fillna('Unknown')

In [ ]:
la_data['LOCATION START DATE'].isna().sum()

In [ ]:
la_data.isna().sum()

In [ ]:
la_data.to_csv('la_data.csv')

In [ ]:
# Since we want to integrate these data with crime data and the crime data is from 2020 to 2021.
# We would go ahead and filter the business with the start date below 2022-01-01

In [ ]:
la_data['LOCATION START DATE'] = pd.to_datetime(la_data['LOCATION START DATE'])

In [ ]:
besiness_below_2022 = la_data.loc[la_data['LOCATION START DATE']<'2022-01-01']

In [ ]:
active_business_cnt = besiness_below_2022['CITY'].value_counts().reset_index()

In [ ]:
active_business_cnt.columns = ['City', 'Cnt']

In [ ]:
active_business_cnt.to_csv('active_business_cnt.csv')

In [ ]:
active_business_cnt

In [ ]:
#https://public.tableau.com/app/profile/shouwang.zhang/viz/ActiveBusinessInLA/Dashboard1?publish=yes